In [13]:
from ipyleaflet import Map, DrawControl, Rectangle
import pandas as pd

df = pd.DataFrame(columns=['southwest', 'northeast'])

m = Map(center=(40, -100), zoom=4)

draw_control = DrawControl(
    rectangle={"shapeOptions": {"color": "#0000FF"}},
    polyline={}, polygon={}, circle={}, circlemarker={}, marker={}
)

def handle_draw(target, action, geo_json):
    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        coords = geo_json['geometry']['coordinates'][0]
        lats = [c[1] for c in coords]
        lons = [c[0] for c in coords]
        sw = (min(lats), min(lons))
        ne = (max(lats), max(lons))
        df.loc[len(df)] = [sw, ne]
        print(f"Rectangle added: SW={sw}, NE={ne}")
        print(df)
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

m

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…